In [ ]:
def forward(self, x, temb):
    # 1. 입력값 보존 (Residual Connection을 위해 x를 보관)
    h = x

    # 2. 첫 번째 블록: GroupNorm -> 활성화 함수 -> 3x3 Convolution
    h = self.norm1(h)
    h = nonlinearity(h)
    h = self.conv1(h)

    # 3. Time Embedding 주입 (매우 중요)
    # temb가 있을 경우, 이를 선형 변환(temb_proj)한 뒤
    # [B, C, 1, 1] 형태로 차원을 확장하여 피처맵 h에 더해줍니다.
    # 이를 통해 모델이 현재 "몇 번째 디퓨전 스텝"인지 인지하게 됩니다.
    if temb is not None:
        h = h + self.temb_proj(nonlinearity(temb))[:,:,None,None]

    # 4. 두 번째 블록: GroupNorm -> 활성화 함수 -> Dropout -> 3x3 Convolution
    h = self.norm2(h)
    h = nonlinearity(h)
    h = self.dropout(h)
    h = self.conv2(h)

    # 5. Skip Connection (Shortcut) 매칭
    # 입력(x)과 출력(h)의 채널 수가 다를 경우, x의 채널 수를 h와 맞춰줍니다.
    if self.in_channels != self.out_channels:
        if self.use_conv_shortcut:
            x = self.conv_shortcut(x) # 3x3 conv 등으로 채널 변경
        else:
            x = self.nin_shortcut(x)  # 1x1 conv 등으로 채널 변경

    # 6. 최종 결과: 입력값(x)과 변환된 값(h)을 더함 (잔차 연결)
    return x + h

In [ ]:
def forward(self, x, t=None, context=None):
    """
    Diffusion U-Net의 전체 Forward Pass
    x: 입력 이미지 (Noisy Image)
    t: 타임스텝 (현재 노이즈 단계)
    context: 추가 조건부 정보 (Optional)
    """

    # 1. 컨텍스트 결합: 추가 정보가 있다면 채널 방향으로 합침
    if context is not None:
        x = torch.cat((x, context), dim=1)

    # 2. Time Embedding 생성: 타임스텝 t를 고차원 벡터로 변환하여 모델이 "단계"를 인식하게 함
    if self.use_timestep:
        assert t is not None
        temb = get_timestep_embedding(t, self.ch)
        temb = self.temb.dense[0](temb)
        temb = nonlinearity(temb)
        temb = self.temb.dense[1](temb) # 최종 Time Embedding (temb)
    else:
        temb = None

    # --- [Downsampling 구간: 이미지를 줄이며 특징 추출] ---
    hs = [self.conv_in(x)] # Skip Connection을 위해 중간 결과들을 저장할 리스트
    for i_level in range(self.num_resolutions):
        for i_block in range(self.num_res_blocks):
            # ResNet 블록 내부에 temb를 주입하여 시간 정보를 반영
            h = self.down[i_level].block[i_block](hs[-1], temb)
            if len(self.down[i_level].attn) > 0:
                h = self.down[i_level].attn[i_block](h) # Self-Attention 적용
            hs.append(h) # 결과값 저장

        # 해상도 축소 (Downsample)
        if i_level != self.num_resolutions-1:
            hs.append(self.down[i_level].downsample(hs[-1]))

    # --- [Middle 구간: 가장 깊은 곳에서의 처리] ---
    h = hs[-1]
    h = self.mid.block_1(h, temb) # ResNet 블록 1
    h = self.mid.attn_1(h)        # Attention
    h = self.mid.block_2(h, temb) # ResNet 블록 2

    # --- [Upsampling 구간: 이미지를 복원하며 Skip Connection 결합] ---
    for i_level in reversed(range(self.num_resolutions)):
        for i_block in range(self.num_res_blocks+1):
            # hs.pop(): Downsampling 때 저장한 피처맵을 뒤에서부터 꺼내 현재 층과 결합(Concat)
            # 이를 통해 소실된 세부 공간 정보를 보충함
            h = self.up[i_level].block[i_block](
                torch.cat([h, hs.pop()], dim=1), temb)
            if len(self.up[i_level].attn) > 0:
                h = self.up[i_level].attn[i_block](h)

        # 해상도 확대 (Upsample)
        if i_level != 0:
            h = self.up[i_level].upsample(h)

    return h # 최종 노이즈 예측값 혹은 복원 이미지 반환